In [ ]:
# 微调
# 我们通常使用到的数据集合在Fashion-MNIST（6万）和ImageNet（1000万）之间
# ImageNet数据集成本花费数百万美元
# 另外一种解决方法是应用迁移学习（transfer learning）
# 将从源数据学习到的知识迁移到目标数据集上虽然ImageNet数据集的图像大多与
# 椅子无关但是模型对于图像特征的抽取能力是同样有效的
# 本节将介绍一种常用的技术： 微调 fine tuning

# fine tuning 由以下四步组成
# （1）在源数据集上（如ImageNet）上训练一个神经网络模型即源模型
# （2）创建一个新的神经网络模型，即目标模型，他复制了源模型上除了输出层之外的所有
# 模型设计及其参数，假设源模型的输出层和源数据集的标签密切相关，因此在目标模型中不予采用
# （3）为目标模型添加一个输出大小为目标数据集类别个数的输出层，并随机初始化该层的模型参数
# （4）在目标数据集（如椅子）上训练目标模型，将从头训练输出层，其余层的参数都是基于源模型
# 参数微调得到的

# 热狗识别
# 我们将基于一个小的数据集对在ImageNet数据集上训练好的ResNet模型进行微调，该小数据集
# 含有数千张包含热狗和不包含热狗的图像，我们将使用微调得到的模型来识别一张图像中是否包含
# 热狗

# 首先导入实验包和模块 
%matplotlib inline
import torch
from torch.utils.data import Dataseta,DataLoader
import torchvision
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torchvision import models
import os

import sys
sys.path.append("..")
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 获取数据集
# 数据集合包含1400张热狗图和1400张非热狗图，各类的1000张图像用于训练，其它用于测试

data_dir = '/s1/CSCL/tangss/Datasets'
os.listdir(os.path.join(data_dir,"hotdog"))#['train','test']
train_imgs = ImageFolder(os.path.join(data_dir,'hotdog/train'))
test_imgs = ImageFolder(os.path.join(data_dir,'hotdog/test'))

hotdogs = [train_imgs[i][0] for i in range(8)]
not_hotdogs = [train_imgs[-i-1][0] for i in range(8)]
d2l.show_images(hotdogs+not_hotdogs,2,8,scale=1.4)

# 指定RGB三个通道的均值和方差来将图像通道归一化
normalize = transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
train_augs = transforms.Compose([
    transforms.RandomResizedCrop(size=224),# 改变像素
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
])

test_augs = transforms.Compose([
    transforms.Resize(size=256),
    transforms.CenterCrop(size=224),
    transforms.ToTensor(),
    normalize
])

# 定义和初始化模型
# 我们使用在ImageNet上预训练的RestNet-18作为源模型
pretrained_net = models.resnet18(pretrained=True)
pretrained_net.fc = nn.Linear(512,2)

# 源模型中除了fc层之外的其它层的参数得以保留，fc层需要训练
output_params = list(map(id,pretrained_net.fc.parameters()))
feature_params = filter(lambda p:id(p) not in output_params,pretrained_net.parameters())

lr = 0.01
optimizer = optim.SGD([
    {'params':feature_params},
    {'params':pretrained_net.fc.parameters(),'lr':lr*10}
],lr=lr,weight_decay=0.001)


# 微调模型
def train_fine_tuning(net,optimizer,batch_size=128,num_epochs=5):
    train_iter = DataLoader(ImageFolder(os.path.join(data_dir,'hotdog/train'),transform=train_augs),batch_size,shuffle=True)
    test_iter = DataLoader(ImageFolder(os.path.join(data_dir,'hotdog/test'),transform=test_augs),batch_size=,shuffle=True)
    loss = torch.nn.CrossEntropLoss()
    d2l.train(train_iter,test_iter,net,loss,optimizer,device,num_epochs)
    
train_fine_tuning(pretrained_net,optimizer)


# 迁移学习将从源数据集学到的知识迁移到目标数据集上，微调是迁移学习的一种常用技术。
# 目标模型的输出层需要从头训练其他不需要
# 一般来说，微调参数会使用较小的学习率，而从头训练输出层可以使用较大的学习率

